In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import os
from random import shuffle
from sklearn.preprocessing import LabelEncoder
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
import conv_blocks as op
import multiprocessing
import mobilenet_v2
import mobilenet
tf.__version__

'1.5.0'

In [2]:
batch_size = 16
epoch = 1000
learning_rate = 1e-3
DIR = 'crop/'

In [3]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def load_image(addr):
    
    return img.astype(np.float32)

def convert_to(dataset, labels_age, labels_gender, name, i):
    if not os.path.exists(name):
        os.mkdir(name)
    filename = os.path.join(name,'file-%d.tfrecords'%(i))
    print('writing %s, cpu %d'%(filename,i))
    with tf.python_io.TFRecordWriter(filename) as writer:
        for index in range(dataset.shape[0]):
            img = imresize(imread(dataset[index]),(224,224)).reshape((224,224,1)).astype(np.float32)
            example = tf.train.Example(features=tf.train.Features(feature={
                'gender': _int64_feature(int(labels_gender[index])),
                'age': _int64_feature(int(labels_age[index])),
                'image':_bytes_feature(tf.compat.as_bytes(img.tostring()))}))
            writer.write(example.SerializeToString())

In [4]:
img_lists = os.listdir(DIR)
shuffle(img_lists)
img_labels, age_labels, images = [], [], []
for i in img_lists:
    splitted = i.split('--')
    if int(splitted[1]) > 100:
        continue
    img_labels.append(splitted[0])
    age_labels.append(int(splitted[1]))
    images.append(DIR+i)
img_Y = LabelEncoder().fit_transform(img_labels)
img_lists = images
img_lists = np.array(img_lists)
age_labels = np.array(age_labels)
img_Y = np.array(img_Y)

In [5]:
cpu_cores = 4
train_idx = np.linspace(0, img_lists.shape[0], cpu_cores + 1, dtype=np.int)
pool = multiprocessing.Pool(processes=cpu_cores)

for p in range(cpu_cores):
    pool.apply_async(convert_to,
                (img_lists[train_idx[p]:train_idx[p + 1] - 1], 
                 age_labels[train_idx[p]:train_idx[p + 1] - 1],
                 img_Y[train_idx[p]:train_idx[p + 1] - 1],
                 'train', p,))
    
pool.close()
pool.join()

In [6]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={'gender': tf.FixedLenFeature([], tf.int64),
                'age': tf.FixedLenFeature([], tf.int64),
                 'image': tf.FixedLenFeature([], tf.string)})
    image = tf.decode_raw(features['image'], tf.float32)
    image = tf.reshape(image, [224,224,1])
    gender = tf.cast(features['gender'], tf.int32)
    age = tf.cast(features['age'], tf.int32)
    return image, gender, age

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

filename_queue = tf.train.string_input_producer(['train/'+i for i in os.listdir('train')], num_epochs=epoch)
img, gender, age = read_and_decode(filename_queue)
X, Y_gender, Y_age = tf.train.shuffle_batch([img, gender, age], batch_size=batch_size, num_threads=6,
                                        capacity=10000,
                                        min_after_dequeue=1000, allow_smaller_final_batch=False)

images = tf.image.grayscale_to_rgb(X)
images = images / 128. - 1
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
    logits, endpoints = mobilenet_v2.mobilenet(images)
logits = tf.nn.relu6(logits)
emotion_logits = slim.fully_connected(logits, 7, activation_fn=None,
                                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                      weights_regularizer=slim.l2_regularizer(1e-5),
                                      scope='emo/emotion_1', reuse=False)

with tf.variable_scope("age"):
    age1=op.expanded_conv(endpoints['layer_16'],160,stride=1)
    age2=op.expanded_conv(age1,320,stride=1)
    age3=mobilenet.global_pool(op.expanded_conv(age2,1280,stride=1,kernel_size=[1, 1]))
    age_logits=slim.conv2d(age3,101, [1, 1],activation_fn=None,normalizer_fn=None,
                biases_initializer=tf.zeros_initializer(),scope='Conv2d_1c_1x1')
    age_logits = tf.squeeze(age_logits, [1, 2])
    
with tf.variable_scope("gender"):
    gender1=op.expanded_conv(endpoints['layer_16'],160,stride=1)
    gender2=op.expanded_conv(gender1,320,stride=1)
    gender3=mobilenet.global_pool(op.expanded_conv(gender2,1280,stride=1,kernel_size=[1, 1]))
    gender_logits=slim.conv2d(gender3,2, [1, 1],activation_fn=None,normalizer_fn=None,
                biases_initializer=tf.zeros_initializer(),scope='Conv2d_1c_1x1')
    gender_logits = tf.squeeze(gender_logits, [1, 2])

gender_accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(gender_logits, Y_gender, 1), tf.float32))
age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)
abs_loss = tf.reduce_mean(tf.abs(tf.cast(Y_age,tf.float32)-age))

global_step = tf.Variable(0, name="global_step", trainable=False)
lr = tf.train.exponential_decay(learning_rate, global_step=global_step, 
                                decay_steps=5000, decay_rate=0.9,staircase=True)

age_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_age, logits=age_logits)
age_cross_entropy_mean = tf.reduce_mean(age_cross_entropy)
gender_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_gender, logits=gender_logits)
gender_cross_entropy_mean = tf.reduce_mean(gender_cross_entropy)
cost = tf.add_n([gender_cross_entropy_mean, abs_loss])
list_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,'age') + tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,'gender')

print(list_vars)
tf.summary.scalar("gender_cross_entropy", gender_cross_entropy_mean)
tf.summary.scalar("total_loss", cost)
tf.summary.scalar("train_abs_age_error", abs_loss)
tf.summary.scalar("gender_accuracy", gender_accuracy)
tf.summary.scalar("learning_rate", lr)
optimizer = tf.train.AdamOptimizer(lr).minimize(cost, var_list=list_vars,global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(init_op)

var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'MobilenetV2') + tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'emo')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, "checkpoints/emotion-checkpoint-mobilenet.ckpt")
saver = tf.train.Saver(tf.global_variables())
saver.save(sess, "checkpoints/combine-checkpoint-mobilenet.ckpt")

[<tf.Variable 'age/expanded_conv/expand/weights:0' shape=(1, 1, 160, 960) dtype=float32_ref>, <tf.Variable 'age/expanded_conv/expand/biases:0' shape=(960,) dtype=float32_ref>, <tf.Variable 'age/expanded_conv/depthwise/depthwise_weights:0' shape=(3, 3, 960, 1) dtype=float32_ref>, <tf.Variable 'age/expanded_conv/depthwise/biases:0' shape=(960,) dtype=float32_ref>, <tf.Variable 'age/expanded_conv/project/weights:0' shape=(1, 1, 960, 160) dtype=float32_ref>, <tf.Variable 'age/expanded_conv/project/biases:0' shape=(160,) dtype=float32_ref>, <tf.Variable 'age/expanded_conv_1/expand/weights:0' shape=(1, 1, 160, 960) dtype=float32_ref>, <tf.Variable 'age/expanded_conv_1/expand/biases:0' shape=(960,) dtype=float32_ref>, <tf.Variable 'age/expanded_conv_1/depthwise/depthwise_weights:0' shape=(3, 3, 960, 1) dtype=float32_ref>, <tf.Variable 'age/expanded_conv_1/depthwise/biases:0' shape=(960,) dtype=float32_ref>, <tf.Variable 'age/expanded_conv_1/project/weights:0' shape=(1, 1, 960, 320) dtype=floa

'new/combine-checkpoint-mobilenet.ckpt'

In [8]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [9]:
try:
    step = sess.run(global_step)
    while not coord.should_stop():
        acc_gender, age_loss,loss,_ = sess.run([gender_accuracy, abs_loss, cost, optimizer])
        summary = sess.run(merged)
        writer.add_summary(summary, global_step=step)
        if step % 200 == 0:
            print('step %d, loss %f, gender accuracy %f, age diff %f'%(step,loss,acc_gender, age_loss))
            saver.save(sess, "checkpoints/combine-checkpoint-mobilenet.ckpt")
        step = sess.run(global_step)
except tf.errors.OutOfRangeError:
    print('Done training for %d epochs, %d steps.' % (epoch, step))
finally:
    coord.request_stop()
coord.join(threads)

step 0, loss 22.192686, gender accuracy 0.437500, age diff 21.499508
step 200, loss 14.507321, gender accuracy 0.875000, age diff 14.128583
step 400, loss 8.341151, gender accuracy 0.812500, age diff 7.891661
step 600, loss 14.059522, gender accuracy 0.875000, age diff 13.778696
step 800, loss 9.018284, gender accuracy 0.875000, age diff 8.561370
step 1000, loss 14.856874, gender accuracy 0.812500, age diff 14.456733
step 1200, loss 12.615247, gender accuracy 0.750000, age diff 12.164854
step 1400, loss 8.732638, gender accuracy 0.937500, age diff 8.429897
step 1600, loss 12.108310, gender accuracy 0.875000, age diff 11.759733
step 1800, loss 14.477577, gender accuracy 0.937500, age diff 14.239824
step 2000, loss 11.499493, gender accuracy 0.937500, age diff 11.329531
step 2200, loss 8.749978, gender accuracy 0.812500, age diff 8.338078
step 2400, loss 12.379673, gender accuracy 0.937500, age diff 11.986933
step 2600, loss 12.786985, gender accuracy 0.750000, age diff 12.356604
step 28

step 23000, loss 10.232597, gender accuracy 1.000000, age diff 10.158941
step 23200, loss 9.113963, gender accuracy 0.937500, age diff 8.900523
step 23400, loss 9.919561, gender accuracy 0.875000, age diff 9.503361
step 23600, loss 9.743428, gender accuracy 0.937500, age diff 9.477063
step 23800, loss 10.177210, gender accuracy 0.875000, age diff 9.766184
step 24000, loss 7.720944, gender accuracy 0.875000, age diff 7.451595
step 24200, loss 11.162285, gender accuracy 0.937500, age diff 10.889971
step 24400, loss 7.765008, gender accuracy 0.812500, age diff 7.475551
step 24600, loss 10.725598, gender accuracy 1.000000, age diff 10.645592
step 24800, loss 8.391574, gender accuracy 0.875000, age diff 8.136404
step 25000, loss 6.444004, gender accuracy 0.875000, age diff 6.272488
step 25200, loss 6.999681, gender accuracy 0.937500, age diff 6.907752
step 25400, loss 10.738687, gender accuracy 1.000000, age diff 10.705489
step 25600, loss 9.786022, gender accuracy 0.875000, age diff 9.4938

step 46000, loss 9.691566, gender accuracy 0.937500, age diff 9.623816
step 46200, loss 7.827639, gender accuracy 0.812500, age diff 7.437222
step 46400, loss 9.499778, gender accuracy 0.937500, age diff 9.353777
step 46600, loss 9.717016, gender accuracy 1.000000, age diff 9.633421
step 46800, loss 9.742319, gender accuracy 0.875000, age diff 9.521626
step 47000, loss 10.434070, gender accuracy 1.000000, age diff 10.379274
step 47200, loss 8.578937, gender accuracy 0.937500, age diff 8.474881
step 47400, loss 8.644316, gender accuracy 1.000000, age diff 8.633080
step 47600, loss 11.010694, gender accuracy 1.000000, age diff 10.869202
step 47800, loss 10.045769, gender accuracy 0.937500, age diff 9.943533
step 48000, loss 9.499102, gender accuracy 1.000000, age diff 9.407628
step 48200, loss 8.974433, gender accuracy 0.875000, age diff 8.696869
step 48400, loss 6.980006, gender accuracy 0.937500, age diff 6.847468
step 48600, loss 8.420555, gender accuracy 0.937500, age diff 8.209229
s

step 69000, loss 8.104418, gender accuracy 0.875000, age diff 7.787526
step 69200, loss 7.490416, gender accuracy 1.000000, age diff 7.450109
step 69400, loss 7.623877, gender accuracy 1.000000, age diff 7.601949
step 69600, loss 9.306555, gender accuracy 0.875000, age diff 9.058871
step 69800, loss 8.773368, gender accuracy 1.000000, age diff 8.746084
step 70000, loss 6.332264, gender accuracy 1.000000, age diff 6.296140
step 70200, loss 9.331511, gender accuracy 0.937500, age diff 9.176203
step 70400, loss 8.718819, gender accuracy 0.875000, age diff 8.431681
step 70600, loss 8.327847, gender accuracy 1.000000, age diff 8.273245
step 70800, loss 8.698475, gender accuracy 0.937500, age diff 8.460566
step 71000, loss 6.783820, gender accuracy 0.937500, age diff 6.682073
step 71200, loss 7.230310, gender accuracy 1.000000, age diff 7.151170
step 71400, loss 6.130629, gender accuracy 0.937500, age diff 6.067943
step 71600, loss 7.964658, gender accuracy 1.000000, age diff 7.918936
step 7

step 92000, loss 8.855661, gender accuracy 0.937500, age diff 8.764215
step 92200, loss 12.185908, gender accuracy 1.000000, age diff 12.131969
step 92400, loss 6.905859, gender accuracy 0.937500, age diff 6.770309
step 92600, loss 6.295298, gender accuracy 0.937500, age diff 6.217623
step 92800, loss 6.736632, gender accuracy 1.000000, age diff 6.699733
step 93000, loss 8.034398, gender accuracy 1.000000, age diff 7.988052
step 93200, loss 7.431229, gender accuracy 1.000000, age diff 7.378236
step 93400, loss 8.856481, gender accuracy 1.000000, age diff 8.814498
step 93600, loss 7.315168, gender accuracy 0.875000, age diff 7.003119
step 93800, loss 7.883167, gender accuracy 1.000000, age diff 7.879380
step 94000, loss 7.794802, gender accuracy 1.000000, age diff 7.781798
step 94200, loss 7.506242, gender accuracy 1.000000, age diff 7.481086
step 94400, loss 8.497822, gender accuracy 0.937500, age diff 8.394861
step 94600, loss 6.447110, gender accuracy 1.000000, age diff 6.438249
step

...

step 228800, loss 7.311576, gender accuracy 1.000000, age diff 7.308314
step 229000, loss 6.817112, gender accuracy 0.937500, age diff 6.691503
step 229200, loss 8.376264, gender accuracy 1.000000, age diff 8.333528
step 229400, loss 7.617914, gender accuracy 1.000000, age diff 7.607024
step 229600, loss 8.252808, gender accuracy 1.000000, age diff 8.245115
step 229800, loss 7.283063, gender accuracy 1.000000, age diff 7.277230
step 230000, loss 6.912627, gender accuracy 1.000000, age diff 6.876839
step 230200, loss 7.810472, gender accuracy 1.000000, age diff 7.786442
step 230400, loss 7.506257, gender accuracy 1.000000, age diff 7.495827
step 230600, loss 8.289922, gender accuracy 0.937500, age diff 8.155134
step 230800, loss 8.722666, gender accuracy 1.000000, age diff 8.716920
step 231000, loss 8.938951, gender accuracy 1.000000, age diff 8.933043
step 231200, loss 9.180080, gender accuracy 1.000000, age diff 9.133675
step 231400, loss 8.399033, gender accuracy 0.937500, age diff 8

step 251600, loss 9.177280, gender accuracy 1.000000, age diff 9.174450
step 251800, loss 6.143126, gender accuracy 0.937500, age diff 6.045507
step 252000, loss 4.954158, gender accuracy 0.937500, age diff 4.820352
step 252200, loss 7.685936, gender accuracy 0.937500, age diff 7.634644
step 252400, loss 7.922551, gender accuracy 1.000000, age diff 7.914960
step 252600, loss 9.099501, gender accuracy 1.000000, age diff 9.089077
step 252800, loss 5.848606, gender accuracy 0.937500, age diff 5.745379
step 253000, loss 8.400634, gender accuracy 1.000000, age diff 8.396500
step 253200, loss 8.137135, gender accuracy 0.937500, age diff 8.053553
step 253400, loss 8.413190, gender accuracy 1.000000, age diff 8.381584
step 253600, loss 7.188932, gender accuracy 1.000000, age diff 7.173276
step 253800, loss 7.964612, gender accuracy 0.875000, age diff 7.816613
step 254000, loss 9.131379, gender accuracy 1.000000, age diff 9.086411
step 254200, loss 6.107254, gender accuracy 1.000000, age diff 6

step 274400, loss 6.670650, gender accuracy 0.937500, age diff 6.604142
step 274600, loss 7.659454, gender accuracy 1.000000, age diff 7.644194
step 274800, loss 9.573100, gender accuracy 1.000000, age diff 9.571390
step 275000, loss 8.796489, gender accuracy 1.000000, age diff 8.768267
step 275200, loss 9.257168, gender accuracy 1.000000, age diff 9.254782
step 275400, loss 9.707297, gender accuracy 1.000000, age diff 9.697639
step 275600, loss 10.059240, gender accuracy 1.000000, age diff 10.049588
step 275800, loss 9.057195, gender accuracy 1.000000, age diff 9.055000
step 276000, loss 7.462698, gender accuracy 1.000000, age diff 7.459299
step 276200, loss 9.889241, gender accuracy 1.000000, age diff 9.831132
step 276400, loss 7.895123, gender accuracy 1.000000, age diff 7.881807
step 276600, loss 8.012107, gender accuracy 1.000000, age diff 8.006328
step 276800, loss 7.055398, gender accuracy 1.000000, age diff 7.052896
step 277000, loss 7.275758, gender accuracy 1.000000, age diff

step 297200, loss 10.061082, gender accuracy 0.937500, age diff 9.942039
step 297400, loss 8.756095, gender accuracy 1.000000, age diff 8.754551
step 297600, loss 7.435338, gender accuracy 1.000000, age diff 7.421965
step 297800, loss 8.586569, gender accuracy 0.875000, age diff 8.367607
step 298000, loss 12.294220, gender accuracy 0.937500, age diff 12.212612
step 298200, loss 8.155967, gender accuracy 1.000000, age diff 8.143949
step 298400, loss 10.268917, gender accuracy 1.000000, age diff 10.260633
step 298600, loss 7.121571, gender accuracy 1.000000, age diff 7.106814
step 298800, loss 7.681357, gender accuracy 1.000000, age diff 7.670802
step 299000, loss 8.880280, gender accuracy 1.000000, age diff 8.878162
step 299200, loss 6.614847, gender accuracy 1.000000, age diff 6.579890
step 299400, loss 9.254182, gender accuracy 1.000000, age diff 9.212382
step 299600, loss 7.613643, gender accuracy 0.937500, age diff 7.532262
step 299800, loss 8.637527, gender accuracy 0.875000, age d

KeyboardInterrupt: 